In [1]:
from m3_functions import *

# Get onset times
# song_path = r'X:\Training Data\Unprocessed\Angevil Hero II\1. John 5 - 27 Needles\song.ogg'
song_path = r'X:\Training Data\Unprocessed\Angevil Hero II\5. Joe Satriani - Big Bad Moon\song.ogg'
_, onset_times = onset_time(song_path)

In [5]:
onsets = onset_time_bins(onset_times)

In [10]:
len(onset_times)
print(onset_times)

[  2.13623583   2.62385488   2.78639456 ... 311.7046712  311.98331066
 312.54058957]


In [13]:
diff = []
for i in range(1, len(onsets)):
    diff.append(onsets[i] - onsets[i-1])

diff = np.array(diff)
largest = diff.argsort()[-10:][::1]
print(diff[largest])

[102 105 123 123 125 133 146 153 164 332]


In [ ]:
# Using viterbi decode

# Load and modify transition table
A = np.load('trans_prob_table.npy')
A_redistributed = redistribute_trans_table(A)

onset = [0 for _ in range(1000)]

B = make_emission_table()
sequence = onset_label(onset)

note_sequence, v, bp = viterbi(A_redistributed, B, sequence)


In [18]:
def create_notes_array(onsets, notes):
    '''
    Takes equal length sequences of onsets and notes and creates a notes array which can be used to create
    final song output (leveraging Model_1/Processing/m1_postprocessing.py -> write_song_from_notes_array())

    ~~~~ ARGUMENTS ~~~~
    - onsets : list
        - contains onset times as 10ms bins
        - should be output of onset_time_bins()
    
    - notes : list or numpy array
        - notes corresponding to the onsets
    '''
    if type(notes) is np.ndarray:
        notes = notes.tolist()

    assert len(onsets) <= len(notes), 'ERROR: There are more onsets than notes'

    # Cut down notes if there are more notes than onsetse
    if len(notes) > len(onsets):
        notes = notes[:(len(onsets))]

    notes_array = np.zeros(onsets[-1]+1)

    np.put(notes_array, onsets, notes)

    return notes_array

# notes_array_viterbi = create_notes_array(onsets, note_sequence)
notes_array_forrest = generate_notes(onsets, interval_length=50)

In [24]:
notes_array = create_notes_array(onsets[:-1], notes_array_forrest)

In [27]:
import sys
sys.path.insert(1, r'C:\Users\ewais\Documents\GitHub\tensor-hero\Model_1\Processing')
from m1_postprocessing import *
from pathlib import Path
import shutil

song_metadata = {'Name' : 'Model 3 - 27 Needles',
                 'Artist' : 'some artist',
                 'Charter' : 'tensorhero',
                 'Offset' : 0,
                 'Resolution' : 192,
                 'Genre' : 'electronic',
                 'MediaType' : 'cd',
                 'MusicStream' : 'song.ogg'}

outfolder = Path(r'C:\Users\ewais\Documents\GitHub\tensor-hero\Model_3\m3_test_1')
write_song_from_notes_array(song_metadata, notes_array, outfolder)
shutil.copyfile(str(song_path), str(outfolder / 'song.ogg'))



'C:\\Users\\ewais\\Documents\\GitHub\\tensor-hero\\Model_3\\m3_test_1\\song.ogg'